In [17]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras import layers
import cv2
from PIL import Image

In [18]:
print(tf.__version__)

2.12.1


In [19]:
# Path to the directory containing the images
data_dir = "Dataset"
correct_filters_dir = os.path.join(data_dir, "correct_filters")
defect_filters_dir = os.path.join(data_dir, "defect_filters")
dataset_paths = ["Dataset/correct_filters", "Dataset/defect_filters"]

In [20]:
# Function to convert PNG images to JPEG
def convert_png_to_jpg(png_image_path):
    image = Image.open(png_image_path)
    jpg_image_path = os.path.splitext(png_image_path)[0] + ".jpg"
    image.convert("RGB").save(jpg_image_path, "JPEG")
    return jpg_image_path

In [21]:
# Convert PNG images to JPEG and delete the PNG ones
for dataset_path in dataset_paths:
    for filename in os.listdir(dataset_path):
        if filename.endswith(".png"):
            png_image_path = os.path.join(dataset_path, filename)
            jpg_image_path = convert_png_to_jpg(png_image_path)
            os.remove(png_image_path)
            print("Converted and deleted:", png_image_path, "->", jpg_image_path)
print("Conversion completed.")

Converted and deleted: Dataset/correct_filters/image_42.png -> Dataset/correct_filters/image_42.jpg
Converted and deleted: Dataset/correct_filters/image_51.png -> Dataset/correct_filters/image_51.jpg
Converted and deleted: Dataset/correct_filters/image_36.png -> Dataset/correct_filters/image_36.jpg
Converted and deleted: Dataset/correct_filters/image_53.png -> Dataset/correct_filters/image_53.jpg
Converted and deleted: Dataset/correct_filters/image_9.png -> Dataset/correct_filters/image_9.jpg
Converted and deleted: Dataset/correct_filters/image_32.png -> Dataset/correct_filters/image_32.jpg
Converted and deleted: Dataset/correct_filters/image_4.png -> Dataset/correct_filters/image_4.jpg
Converted and deleted: Dataset/correct_filters/image_68.png -> Dataset/correct_filters/image_68.jpg
Converted and deleted: Dataset/correct_filters/image_23.png -> Dataset/correct_filters/image_23.jpg
Converted and deleted: Dataset/correct_filters/image_7.png -> Dataset/correct_filters/image_7.jpg
Conver

In [22]:
def load_and_preprocess_image(image_path):
    # Load image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    # Normalize pixel values
    img = tf.cast(img, tf.float32) / 255.0 
    return img

def color_temperature(img):
    # Adjust hue of the image to simulate color temperature change
    img = tf.image.adjust_hue(img, np.random.uniform(0.05, 0.25))  # Adjust this value as needed
    return img

def add_dust(img):
    # Add white noise to the image to simulate dust
    dust = np.random.normal(size=img.shape, scale=np.random.uniform(0.05, 0.25))
    img = tf.clip_by_value(img + dust, 0, 1)  # Clip pixel values to [0, 1]
    return img

def add_reflections(img):
    # Add reflections by overlaying a pattern onto the image
    pattern = cv2.imread('reflection_pattern.jpg', cv2.IMREAD_GRAYSCALE)
    pattern = cv2.resize(pattern, img.shape[:2])
    alpha = 0.3  # Adjust this value as needed
    img = cv2.addWeighted(img, 1 - alpha, pattern, alpha, 0)
    return img

def adjust_saturation(img):
    # Increase the saturation of the image
    img = tf.image.adjust_saturation(img, 3)  # Adjust this value as needed
    return img

def adjust_brightness(img):
    # Increase the brightness of the image
    delta = tf.random.uniform(shape=[], minval=-0.3, maxval=0.3)  # Adjust these values as needed
    img = tf.image.adjust_brightness(img, delta)
    return img

def add_reflections(img):
    # Add reflections by overlaying a pattern onto the image
    pattern_path = 'reflection_pattern.jpg'
    if not os.path.exists(pattern_path):
        raise FileNotFoundError(f"Reflection pattern image not found: {pattern_path}")

    pattern = cv2.imread(pattern_path, cv2.IMREAD_GRAYSCALE)
    pattern = cv2.resize(pattern, (img.shape[1], img.shape[0]))  # Resize pattern to match image dimensions
    pattern = np.expand_dims(pattern, axis=-1)  # Add a channel dimension to the pattern
    pattern = np.repeat(pattern, 3, axis=-1)  # Repeat the pattern across the color channels
    
    alpha = 0.3  # Adjust this value as needed
    img = cv2.addWeighted(img.numpy(), 1 - alpha, pattern, alpha, 0)
    img = tf.convert_to_tensor(img, dtype=tf.float32) / 255.0  # Convert back to tensor and normalize
    return img

In [23]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomContrast(0.2),
    layers.Lambda(adjust_brightness),
    #layers.Lambda(lambda img: tf.py_function(add_reflections, [img], tf.float32))
    #layers.GaussianNoise(0.05),
    #layers.Lambda(color_temperature),
    #layers.Lambda(add_dust),
    #layers.Lambda(adjust_saturation)
])

In [24]:
# Function to apply data augmentation and save the augmented images
def apply_augmentation_and_save(image_path, output_dir, prefix, n_augmentations=1):
    image = load_and_preprocess_image(image_path)
    for i in range(n_augmentations):
        augmented_image = data_augmentation(image)
        augmented_image = tf.image.convert_image_dtype(augmented_image, tf.uint8)
        augmented_image = tf.image.encode_jpeg(augmented_image)
        with tf.io.gfile.GFile(os.path.join(output_dir, f"{prefix}_{i}.jpg"), 'wb') as file:
            file.write(augmented_image.numpy())

In [25]:
# Apply data augmentation and save the augmented images
output_dir_correct = correct_filters_dir
output_dir_defect = defect_filters_dir
os.makedirs(output_dir_correct, exist_ok=True)
os.makedirs(output_dir_defect, exist_ok=True)

In [26]:
# Apply augmentation to images in the "correct_filters" directory
for filename in os.listdir(correct_filters_dir):
    image_path = os.path.join(correct_filters_dir, filename)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        apply_augmentation_and_save(image_path, output_dir_correct, filename.split('.')[0])

In [27]:
# Apply augmentation to images in the "defect_filters" directory
for filename in os.listdir(defect_filters_dir):
    image_path = os.path.join(defect_filters_dir, filename)
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        apply_augmentation_and_save(image_path, output_dir_defect, filename.split('.')[0])